In [ ]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from crud_module import AnimalShelter

# Data Manipulation / Model
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 32266, 'AAC', 'animals')

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

# Dashboard Layout / View
app = JupyterDash(__name__)

# FIX ME Add in Grazioso Salvare’s logo
image_filename = '/home/yonuschiboub_snhu/Desktop/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height': '100px'}),
    html.Center(html.B(html.H1('Yonus Chiboub: CS-340 Dashboard'))),
    html.Hr(),
    dcc.RadioItems(
        id='rescue-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water_rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain_wilderness_rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster_individual_tracking'},
            {'label': 'Reset', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'block'}
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',  # Allows users to select a single row
        selected_rows=[],  # Initializes with no rows selected
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Interaction Between Components / Controller
@app.callback(
    [Output('datatable-id', 'data'), Output('datatable-id', 'columns')],
    [Input('rescue-type', 'value')]
)
def update_dashboard(rescue_type):
    if rescue_type == 'all':
        query = {}
    else:
        # Example filters based on provided specifications
        breed_filters = {
            'water_rescue': ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
            'mountain_wilderness_rescue': ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
            'disaster_individual_tracking': ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
        }
        sex_filters = {
            'water_rescue': "Intact Female",
            'mountain_wilderness_rescue': "Intact Male",
            'disaster_individual_tracking': "Intact Male"
        }
        age_filters = {
            'water_rescue': (26, 156),
            'mountain_wilderness_rescue': (26, 156),
            'disaster_individual_tracking': (20, 300)
        }
        breeds = breed_filters.get(rescue_type, [])
        sex = sex_filters.get(rescue_type, "")
        age_range = age_filters.get(rescue_type, (0, 5200))  # using a large default range if not specified

        query = {
            'breed': {'$in': breeds},
            'sex_upon_outcome': sex,
            'age_upon_outcome_in_weeks': {'$gte': age_range[0], '$lte': age_range[1]}
        }
    
    records = db.read(query)
    df_filtered = pd.DataFrame.from_records(records)
    
    if not df_filtered.empty:
        if '_id' in df_filtered.columns:
            df_filtered.drop(columns=['_id'], inplace=True)

        data = df_filtered.to_dict('records')
        columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_filtered.columns]
    else:
        data = []
        columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]

    return data, columns


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return html.Div("No data available.")
    df = pd.DataFrame.from_dict(viewData)
    return dcc.Graph(figure=px.pie(df, names='breed', title='Preferred Animals'))

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if viewData is None or not selected_rows:
        # Return a default map if no data is available or no row is selected
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ])

    dff = pd.DataFrame.from_dict(viewData)
    # Check if selected_rows list is not empty and then get the row
    if selected_rows:
        row = selected_rows[0]
        if row < len(dff):
            # Extract latitude and longitude based on your DataFrame's column indexing
            latitude = dff.iloc[row, 13]  # Assuming 13th column is latitude
            longitude = dff.iloc[row, 14]  # Assuming 14th column is longitude
            return [
                dl.Map(style={'width': '1000px', 'height': '500px'},
                       center=[latitude, longitude], zoom=10, children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(position=[latitude, longitude], children=[
                        dl.Tooltip(dff.iloc[row, 4]),  # Assuming 4th column is a tooltip field
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row, 9])  # Assuming 9th column is animal name
                        ])
                    ])
                ])
            ]
    # Fallback if no valid row index is found
    return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id")
    ])


app.run_server(debug=True)
